# JSON/YAML Features with Howso Engine

> Note: This feature is experimental and is not universally recommended for production deployment.

## Overview

This notebook provides an overview of using JSON/YAML as features in a Trainee to represent complex data structures in a single feature.

In [1]:
import pandas as pd

from howso.engine import Trainee
from howso.utilities import infer_feature_attributes

## Step 1: Load Data

We use a small, toy-dataset to demonstrate this functionality where each case represents a character in a game.

In [2]:
data = pd.DataFrame([
    {"name": "Nerissa", "level": 14, "class": "Fighter", "inventory": "{\"saddle\": 1, \"trident\": 2, \"potion\": 5}"},
    {"name": "Maaya", "level": 13, "class": "Warlock", "inventory": "{\"sword\": 1, \"lute\": 1, \"potion\": 2}"},
    {"name": "Taxuul", "level": 15, "class": "Cleric", "inventory": "{\"hammer\": 1, \"metal bar\": 20, \"potion\": 1}"},
])

data

,name,level,class,inventory
0,Nerissa,14,Fighter,"{""saddle"": 1, ""trident"": 2, ""potion"": 5}"
1,Maaya,13,Warlock,"{""sword"": 1, ""lute"": 1, ""potion"": 2}"
2,Taxuul,15,Cleric,"{""hammer"": 1, ""metal bar"": 20, ""potion"": 1}"


## Step 2: Define the Feature Attributes

The feature-attributes for the ``inventory`` feature are automatically inferred to be JSON.

In [3]:
features = infer_feature_attributes(data)
features.to_dataframe()

type decimal_places     bounds                           \
                                     allow_null   min   max observed_min   
name          nominal            NaN       True   NaN   NaN          NaN   
level      continuous            0.0      False  12.0  16.0         13.0   
class         nominal            NaN       True   NaN   NaN          NaN   
inventory  continuous            NaN        NaN   NaN   NaN          NaN   

                       data_type original_type       
          observed_max               data_type size  
name               NaN       NaN        string  NaN  
level             15.0    number       integer  8.0  
class              NaN       NaN        string  NaN  
inventory          NaN      json        string  NaN

## Step 3: Train and Analyze

In [4]:
t = Trainee(features=features)
t.train(data)
t.analyze()

## Step 4: React

### Part 1: Discriminative React

First, we use a discriminative `react` to determine a likely inventory.

In [5]:
discriminative_action = t.react(
    contexts=[[15, "Warlock"]], context_features=["level", "class"],
    action_features=["inventory"],
)["action"]

discriminative_action.loc[:, "inventory"]

0    {"lute":1,"potion":4.090399899271196,"saddle":1}
Name: inventory, dtype: object

### Part 2: Generative React

We can also use a generative `react` to generate any number of more or less surprising inventories. We will do this generative `react` with differing levels of desired conviction to show more and less surprising examples.

With higher desired conviction, we will see JSON that more closely resembles the training data.

> Note: `react` is generative when `desired_conviction` is specified. See more about generative vs. discriminative reacts in the "engine_generative_react.ipynb" recipe. Also see the [documentation for the react method](https://docs.howso.com/en/release-latest/api_reference/_autosummary/howso.engine.html#howso.engine.Trainee.react).

In [6]:
generative_reaction_25 = t.react(
    contexts=[[15, "Warlock"]], context_features=["level", "class"],
    action_features=["inventory"],
    desired_conviction=25,
    num_cases_to_generate=5,
)["action"]

generative_reaction_25.loc[:, "inventory"]

0    {"hammer":1,"potion":1,"metal bar":20}
1           {"lute":1,"potion":2,"sword":1}
2           {"lute":1,"potion":2,"sword":1}
3       {"trident":2,"potion":5,"saddle":1}
4       {"trident":2,"potion":5,"saddle":1}
Name: inventory, dtype: object

With a lower desired conviction, we will see JSON that looks very different from the training data.

In [7]:
generative_reaction_1 = t.react(
    contexts=[[15, "Warlock"]], context_features=["level", "class"],
    action_features=["inventory"],
    desired_conviction=1,
    num_cases_to_generate=5,
)["action"]

generative_reaction_1.loc[:, "inventory"]

0                                        {}
1           {"lute":1,"potion":2,"sword":1}
2    {"hammer":1,"potion":1,"metal bar":20}
3       {"trident":2,"potion":5,"saddle":1}
4       {"trident":2,"potion":5,"saddle":1}
Name: inventory, dtype: object

Notably, we may see completely new strings or data types associated with JSON keys that already exist in the dataset. While this isn't a bug, this demonstrates some of the behavior that can be expected when using JSON as a feature especially in generative output from the Howso Engine.

In [8]:
t.delete()

# Conclusion

In this recipe we have shown a small demo of a dataset with a JSON feature. In this example, we have shown the Howso Engine's ability to predict values of a JSON feature effectively as well as do some generative modeling of JSON feature values. 

While it is not shown here, the Howso Engine also has preliminary support for YAML as a feature too.

Lastly, we would like to reiterate that this feature is experimental and is not universally recommended for production deployment.